In [ ]:
#!/usr/bin/python3

import os
import string
import threading
from queue import Queue
import time
import pandas as pd
import numpy as np
from typing import List
from websocket import create_connection
import socket
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderStatus
from alpaca.trading.requests import LimitOrderRequest
from alpaca.trading.requests import GetOrdersRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest

from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator
from ta.trend import VortexIndicator
from ta.volatility import BollingerBands
import time
import datetime as dt

In [2]:
def stream_data(symbol, lookback):
    # date formatting
    start_date = dt.date.today() - dt.timedelta(days=lookback)
    start_date = dt.datetime.combine(start_date, dt.datetime.min.time())
    end_date = dt.date.today()
    end_date = dt.datetime.combine(end_date, dt.datetime.min.time())

    request_params = CryptoBarsRequest(
        symbol_or_symbols=symbol,  # Can be a list of symbols or a single symbol ['BTC/USD', 'ETH/USD'] or 'BTC/USD'
        timeframe=TimeFrame.Minute,
        start=start_date,
    )
    cryptoClient = CryptoHistoricalDataClient()
    data = cryptoClient.get_crypto_bars(request_params)
    data = data.df
    data = data.reset_index(pd.Index(['symbol']))

    # calculate indicators
    data['ema'] = EMAIndicator(data.close, 20).ema_indicator()
    data['rsi'] = RSIIndicator(data.close, 14).rsi()
    bb = BollingerBands(data.close, 20, 2)
    data['bb_upper'] = bb.bollinger_hband()
    data['bb_lower'] = bb.bollinger_lband()
    data['bb_middle'] = bb.bollinger_mavg()

    # replace the NaN values with 0 and return the dataframe
    data.fillna(0, inplace=True)
    return data

def check_positions(symbol):
    trading_client = TradingClient('PKFZSPYZXB1ZDPPZ1UZU', '7KFPD5JYczbvPstNCtbF4GzVrblxBZc00q06LVPx')

    positions = trading_client.get_all_positions()
    # print(positions)
    symbol = symbol.replace('/', '')
    # count the number of positions  with the symbol we are looking for and return the number of positions we
    # have open for that symbol
    return len([position for position in positions if position.symbol == symbol])

# strategy
# first check if we have any positions for the symbol in question
# then check if we have enough cash to buy the asset
# then place the order
# then wait for the order to be filled
def strategy(data, symbol):
    trading_client = TradingClient('PKFZSPYZXB1ZDPPZ1UZU', '7KFPD5JYczbvPstNCtbF4GzVrblxBZc00q06LVPx')
    account = trading_client.get_account()
    cash = float(account.non_marginable_buying_power)
    positions = check_positions(symbol)

    # use 5% of the cash to buy the asset
    cash_to_use = cash * 0.05
    # calculate the number of assets to buy
    qty = cash_to_use / data.close.iloc[-1]
    qty = round(qty, 3)

    # if we have no positions for the symbol in question, and we have enough cash to buy the asset
    if positions == 0 and cash_to_use > 0:

        # buy the asset
        if (data['rsi'].iloc[-1] < 30) and (data['close'].iloc[-1] < data['ema'].iloc[-1]) and (
                data['close'].iloc[-1] < data['close'].iloc[-2]):
            # place the order
            order = trading_client.submit_order(
                MarketOrderRequest(
                    symbol=symbol,
                    qty=qty,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY,
                )
            )
            print('Buy order placed for ' + symbol + ' at ' + str(data.close.iloc[-1]))
            return order
        # check if we have a position for the symbol in question and if we do close it
    elif positions > 0:

        # sell the asset
        if (data['rsi'].iloc[-1] > 70) and (data['close'].iloc[-1] > data['ema'].iloc[-1]) and (
                data['close'].iloc[-1] > data['close'].iloc[-2]):
            # place the order
            order = trading_client.submit_order(
                MarketOrderRequest(
                    symbol=symbol,
                    qty=qty,
                    side=OrderSide.SELL,
                    time_in_force=TimeInForce.DAY,
                )
            )
            print('Sell order placed for ' + symbol + ' at ' + str(data.close.iloc[-1]))
            return order
    else:
        return None

In [ ]:
while True:
